In [1]:
import os

In [2]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one\\Experimentation'

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one'

In [21]:
#  Adding Environment Variables  ( No need to copy this to modular code since we used git bash to export them)

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/swarajbari18/Portfolio_Project_one.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="swarajbari18"
os.environ["MLFLOW_TRACKING_PASSWORD"]="624708f51d3a09f6c16c93c4ba5a1491c9137622"



In [14]:
#  Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [16]:
#  Configuration Manager
from portfolio_project_one.constants import *
from portfolio_project_one.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:                  
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,                     # These were all defined in constants
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifact_root])


    def get_model_evaluation_configuration(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri= "https://dagshub.com/swarajbari18/Portfolio_Project_one.mlflow"
        )
        return model_evaluation_config



In [23]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_metrics(self, actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        return rmse, mae, r2

    def log_metrics_to_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(columns= self.config.target_column)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme   # if we don't have a remote server uri, then this will return file, 
                                                                                # since the mlflow also runs locally
        with mlflow.start_run():

            predictions = model.predict(X_test)
            (rmse, mae, r2) = self.evaluate_metrics(y_test, predictions)

            #  Saving metrics at local machine
            scores = {'rmse': rmse, 'mae': mae, 'r2_score': r2}
            save_json(path= Path(self.config.metric_file_name), data= scores)

            #  logging metrics to mlflow
            mlflow.log_metric("Root Mean Squared Error", rmse)
            mlflow.log_metric("Mean Absolute Error", mae)
            mlflow.log_metric("R2 Square", r2)

            # Model Registry does not work with file store
            if tracking_url_type_store != "file":
                #  Register the model
                #  There are several other ways to use Model Registry as well
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")

            else:
                mlflow.sklearn.log_model(model, "model")

In [24]:
try: 
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_configuration()
    model_evaluation = ModelEvaluation(config= model_evaluation_config)
    model_evaluation.log_metrics_to_mlflow()

except Exception as e:
    raise e

[2023-10-09 16:43:10,131: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-09 16:43:10,135: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-09 16:43:10,139: INFO: common: yaml file: schema.yaml loaded succesfully]
[2023-10-09 16:43:10,141: INFO: common: Created directory at : artifacts]
[2023-10-09 16:43:10,143: INFO: common: Created directory at : artifacts/model_evaluation]
[2023-10-09 16:43:12,221: INFO: common: Saved json file successfully at:artifacts\model_evaluation\metrics.json]


D:\Machine_Learning\Portfolio_Project_one\VirtualEnvironment\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2023/10/09 16:44:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
